# 第十三週：LLM Information Extraction
第二組

1. LLM Inference 基本範例(情緒+關鍵字分類)
2. Prompt Engineering
3. RAG

# 連接雲端資料夾

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

#os.chdir('/content/drive/MyDrive/碩士/碩一下/social_media_analysis/study group 4/week13') #切換該目錄
os.chdir('/content/drive/MyDrive/研究所/SMA/week13') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['requirements.txt', 'dict', 'data', 'LLM_w13.ipynb', 'LLM_w13_2.ipynb']

# 套件安裝

In [ ]:
!pip install jieba

!pip install sentence-transformers
!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# HuggingFace Token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [ ]:
# repo_id: HuggingFace 上的模型名稱，本範例使用 gemma 7b instruct 版本
# huggingface: https://huggingface.co/google/gemma-1.1-7b-it
# temperature: 模型回答的活潑程度，0~1 越大每次回答的多樣性越高

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-1.1-7b-it", temperature=1.0,model_kwargs={'token':HUGGINGFACEHUB_API_TOKEN, 'use_cache':False}
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


# 1. Information Extraction
資料集: 2023/05/20-2024/05/20 PTT「車」版，共1974筆資料。

In [ ]:
df = pd.read_csv("./data/EV_2023_2024.csv")
df["artDate"] = pd.to_datetime(df["artDate"])
df['artDate'] = df['artDate'].dt.date

clear_df = df.copy()

#去除本次不會用到的欄位
drop_cols = ['system_id', 'artPoster', 'artCatagory', 'artComment', 'e_ip', 'insertedDate', 'dataSource']
clear_df.drop(drop_cols, axis = 1, inplace = True)

#去除文章內容為空值的筆數
clear_df.dropna(subset = ['artContent'], axis=0, how='any', inplace=True)

#新增['sentence']欄位，用'。'取代'\n\n'，並移除'\n'
clear_df['sentence'] = clear_df['artContent'].str.replace(r'\n\n','。', regex=True)
clear_df['sentence'] = clear_df['sentence'].str.replace(r'\n','', regex=True)

#移除內文中的網址
clear_df['sentence'] = clear_df['sentence'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

clear_df['sentence'] = clear_df['sentence'].str.split("[,，。！!？?]{1,}")

sent_df = clear_df.explode('sentence').reset_index(drop=True)
# 把長度小於1的sentence刪掉
sent_df = sent_df[sent_df["sentence"].str.len() > 1]

sampled_df = sent_df.sample(n=50, random_state=222)

sampled_df

,artUrl,artTitle,artDate,artContent,sentence
5339,https://www.ptt.cc/bbs/car/M.1687881934.A.112....,[問題]RX350&RX350h抉擇的兩難,2023-06-28,文章在01發文過，票數相當各有擁護者\n想再問問板上的各位前輩意見（鞠躬）\n\n上個月台股...,車室本來就很好
3010,https://www.ptt.cc/bbs/car/M.1686196971.A.73C....,Re:[問題]Nx200350FSprotMODELY,2023-06-08,您列的這幾部車都是好車，但如果硬要比的話，以使用者過來人的經驗提供給您參考。\n\n以下是我...,遠端APP可控制車輛的的便利性
15684,https://www.ptt.cc/bbs/car/M.1696528615.A.91E....,[新聞]幾乎一統美國充電江山！現代與KIA加入特,2023-10-06,原文連結：\nhttps://udn.com/news/amp/story/6811/748...,畢竟在美國特斯拉真的遠超越其他牌
41309,https://www.ptt.cc/bbs/car/M.1710066962.A.A93....,[情報]趙小蘭妹妹生前駕特斯拉墜池塘狂求救,2024-03-10,好可怕怎開特斯拉會這個樣子\n一台兩百多萬到四百多萬的頂級豪華電車耶\n如果她開的是傳統汽油...,享年50歲
15327,https://www.ptt.cc/bbs/car/M.1696396176.A.E42....,Re:[討論]補助60年換來一家代理中國汽車的公司,2023-10-04,:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n政...,法國人可以買走
35883,https://www.ptt.cc/bbs/car/M.1708349881.A.7A0....,[閒聊]2024年國內燃油新車總整理｜歐美篇,2024-02-19,參考新聞來源：\nhttps://www.mobile01.com/topicdetail....,4.Aston Martin DB12 Volante Martin去年正式發表全新世代的DB12
11137,https://www.ptt.cc/bbs/car/M.1691747991.A.A48....,[新聞]中國想藉由投資韓國電池供應鏈，以獲得,2023-08-11,中國想藉由投資韓國電池供應鏈，以獲得美國IRA法案補貼\n\n原文連結：\nhttps://...,中國擁有剛果的鈷礦和印度尼西亞的鎳礦等海外採礦業務
5417,https://www.ptt.cc/bbs/car/M.1688023859.A.0D9....,[新聞]馬斯克一語成讖！美新創電動車申請破產,2023-06-29,原文連結：\nhttps://auto.ltn.com.tw/news/23176/\n特斯...,2020 年正式上市
23954,https://www.ptt.cc/bbs/car/M.1701002913.A.7ED....,[心得]Mercedes-BenzW206C180一年駕駛心得,2023-11-26,（文長）\n從分享開箱心得到現在，陸續都有收到版友詢問使用上的心得，\n所以除了想分享一年多...,100%市區不大腳行駛下大約是10KM/L
10850,https://www.ptt.cc/bbs/car/M.1691550798.A.A5D....,Re:[討論]高里程該選油車還是電車?,2023-08-09,我是油車直接換電車，因公司的車位是機械，所以當時選的是MODEL 3，開了八個月，\n\n簡...,有些人算到一年差幾萬元


這邊我們特別取5篇文章出來，看各篇文章的總情緒。

In [ ]:
# 隨機選出5個不同的 artTitle
random_titles = sent_df['artTitle'].drop_duplicates().sample(n=5, random_state=2)

# 根據這5個隨機選出的 artTitle 找出相應的資料
sampled_5article_df = sent_df[sent_df['artTitle'].isin(random_titles)]

# 顯示結果
sampled_5article_df

,artUrl,artTitle,artDate,artContent,sentence
5040,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,我朋友以前常常去水牛公園超級充電超級充電
5041,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,大概750元
5042,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,太貴
5043,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,在嘉義火車站文化園區充電10小時400元還是太貴
5044,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,後來我朋友就發現文化創意產業園區有消費的話停車費半價
...,...,...,...,...,...
43560,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,包含電動車組件跟相關系統
43561,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,粉專有預先登記入場的連結
43562,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,到時候可以掃QR code入場 馳諾瓦(飛宏子公司)
43563,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,OTA部分科絡達好似蠻強大


下面的車廠是找找2023 Q2全球十大電動車廠的前8名，來自[yahoo新聞](https://tw.news.yahoo.com/%E5%85%A8%E7%90%83%E5%8D%81%E5%A4%A7%E9%9B%BB%E5%8B%95%E8%BB%8A%E5%BB%A0%E6%8E%92%E5%90%8D%E5%87%BA%E7%88%90-%E7%89%B9%E6%96%AF%E6%8B%89-%E6%AF%94%E4%BA%9E%E8%BF%AA%E5%88%86%E5%B1%85%E5%86%A0%E4%BA%9E%E8%BB%8D-%E4%BD%86%E7%AC%AC%E4%B8%89%E5%90%8D%E4%BD%A0%E4%B8%8D-%E5%AE%9A%E7%8C%9C%E7%9A%84%E5%88%B0-073520879.html?guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAJi-up5YbiLexZrnOPje_iLwpOclbB_4fQ7719U2RUPrEIQgtUBNKSzkJcP3w5Nw3GhHjGAQK2DbQxJR1Lz0l9q_qjYqDecLjbUOKHwVr6g6gvwYRYCKyHgndV2a9QuQbV_5VG8M2Uirggv72euASwdI2kc3qifwHwu5x3RoxH4o)

In [ ]:
class EV(BaseModel):
    情緒: str = Field(..., description="判斷整個句子總體的情緒。", enum=["正向", "中性", "負向"])
    電動車品牌: str = Field(
        ...,
        description="判斷句子中提到的電動車是屬於什麼品牌的。",
        enum=["特斯拉", "比亞迪", "廣汽埃安", "福斯", "上汽通用五菱", "現代", "BMW", "賓士", "其他", "無"],
    )

EV_parser = JsonOutputParser(pydantic_object=EV)

# 透過 get_format_instructions() 可以看到如何告訴 LLM 生成我們想要的格式的一些指令
EV_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u500b\\u53e5\\u5b50\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["\\u6b63\\u5411", "\\u4e2d\\u6027", "\\u8ca0\\u5411"], "type": "string"}, "\\u96fb\\u52d5\\u8eca\\u54c1\\u724c": {"title": "\\u96fb\\u52d5\\u8eca\\u54c1\\u724c", "description": "\\u5224\\u65b7\\u53e5\\u5b50\\u4e2d\\u63d0\\u5230\\u7684\\u96fb\\u52d5\\u8eca\\u662f\\u5c6c\\u65bc\\u4ec0\\u9ebc\\u54c1\\u724c\\u7684\\u3002", "enum": ["\\u7279\\u65af\\u62c9", "\\u6bd4\\u4

我們的template

In [ ]:
# Set up a parser + inject instructions into the prompt template.
EV_parser = JsonOutputParser(pydantic_object=EV)\

EV_template = """<start_of_turn>user
你是一位電動車評論家，你將會看網路上對於各品牌電動車的評論，請抓取出評論中：
1.此評論的情緒，正向或負向或中性
2.討論到的電動車品牌，例如：特斯拉

以下為一些範例：
```
範例1
句子：2040年，不再販售燃油車，市面上將只會看到電動車
抓取結果：{{
    "情緒":"中性",
    "電動車品牌":"無"
}}

範例2
句子：身為美國品牌的特斯拉，卻大幅暴跌到第62名，比去年下降50名，成百大品牌聲譽排行榜衰退最多品牌。
抓取結果：{{
    "情緒":"負向",
    "電動車品牌":"特斯拉"
}}

範例3
句子：Kia的車子外觀真的讚讚
抓取結果：{{
    "情緒":"正向",
    "電動車品牌":"其他"
}}

範例4
句子：普普通通
抓取結果：{{
    "情緒":"中性",
    "電動車品牌":"無"
}}


{format_instructions}
請使用繁體中文回答。
除了 JSON 內容外，不要回答任何其他內容

現在請幫我抓取以下句子的資訊：
句子：{query}
抓取結果：<end_of_turn>
<start_of_turn>model
"""

EV_prompt = PromptTemplate(
    template=EV_template,
    # template="Answer the user query.\n{format_instructions}\nDon't output any other explanation\n{query}\n",
    input_variables=["query"],
    # 將 format_instructions 也輸入到 prompt templete 中
    partial_variables={"format_instructions": EV_parser.get_format_instructions()},
)

chain = EV_prompt | llm
chain.get_graph().print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
      +----------------+       
      | PromptTemplate |       
      +----------------+       
              *                
              *                
              *                
   +---------------------+     
   | HuggingFaceEndpoint |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| HuggingFaceEndpointOutput |  
+---------------------------+  


手動建立 parse function

In [ ]:
# 從 json 字串中抓取答案
def extract_answer_from_string(string_):
    string_ = string_.replace("\n", "")
    string_ = string_.replace("json", "")
    string_ = string_.replace("    ", "")
    string_ = string_.replace("```", "")
    try:
        # 可以成功轉換
        dict_result = json.loads(string_)
    except:
        print(string_)
        return string_
    return dict_result

# 從 dict 抓取情緒
def get_sentiment(row):
    sentiment = None
    if isinstance(row, dict):
        if ("情緒" in row.keys()):
            sentiment = row['情緒']
    return sentiment

下面的測試有正確判斷為負向

In [ ]:
llm_result = extract_answer_from_string(chain.invoke({"query": "電動車是要去哪裡充電，好麻煩"}))
print(llm_result)
get_sentiment(llm_result)

{'情緒': '負向', '電動車品牌': '無'}


'負向'

對兩個抽取的資料及做llm情緒判斷

In [ ]:
sampled_5article_df['LLM'] = sampled_5article_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
sampled_df['LLM'] = sampled_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
sampled_df

The provided context does not include the sentence "嚴重的話還會被抓去關…", so I am unable to extract the requested data from the given context.
The provided context does not contain any information regarding opinions or electric car brands, so I am unable to extract the requested data from the given sentence.
The provided text is not included in the prompt, so I am unable to extract the requested information.
資料不足，無法抓取資訊。請提供完整的句子內容。


<ipython-input-14-d7d8b7035606>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_5article_df['LLM'] = sampled_5article_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)


The provided text is not included in the prompt, so I am unable to extract the requested information.
The provided text does not include any sentences regarding the cost of repairs for electric vehicles, so I am unable to extract the requested data from the given context.


,artUrl,artTitle,artDate,artContent,sentence,LLM
5339,https://www.ptt.cc/bbs/car/M.1687881934.A.112....,[問題]RX350&RX350h抉擇的兩難,2023-06-28,文章在01發文過，票數相當各有擁護者\n想再問問板上的各位前輩意見（鞠躬）\n\n上個月台股...,車室本來就很好,"{'情緒': '正向', '電動車品牌': '無'}"
3010,https://www.ptt.cc/bbs/car/M.1686196971.A.73C....,Re:[問題]Nx200350FSprotMODELY,2023-06-08,您列的這幾部車都是好車，但如果硬要比的話，以使用者過來人的經驗提供給您參考。\n\n以下是我...,遠端APP可控制車輛的的便利性,"{'情緒': '正向', '電動車品牌': '無'}"
15684,https://www.ptt.cc/bbs/car/M.1696528615.A.91E....,[新聞]幾乎一統美國充電江山！現代與KIA加入特,2023-10-06,原文連結：\nhttps://udn.com/news/amp/story/6811/748...,畢竟在美國特斯拉真的遠超越其他牌,"{'情緒': '正向', '電動車品牌': '特斯拉'}"
41309,https://www.ptt.cc/bbs/car/M.1710066962.A.A93....,[情報]趙小蘭妹妹生前駕特斯拉墜池塘狂求救,2024-03-10,好可怕怎開特斯拉會這個樣子\n一台兩百多萬到四百多萬的頂級豪華電車耶\n如果她開的是傳統汽油...,享年50歲,"{'情緒': '中性', '電動車品牌': '無'}"
15327,https://www.ptt.cc/bbs/car/M.1696396176.A.E42....,Re:[討論]補助60年換來一家代理中國汽車的公司,2023-10-04,:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n政...,法國人可以買走,"{'情緒': '中性', '電動車品牌': '無'}"
35883,https://www.ptt.cc/bbs/car/M.1708349881.A.7A0....,[閒聊]2024年國內燃油新車總整理｜歐美篇,2024-02-19,參考新聞來源：\nhttps://www.mobile01.com/topicdetail....,4.Aston Martin DB12 Volante Martin去年正式發表全新世代的DB12,"{'情緒': '中性', '電動車品牌': '無'}"
11137,https://www.ptt.cc/bbs/car/M.1691747991.A.A48....,[新聞]中國想藉由投資韓國電池供應鏈，以獲得,2023-08-11,中國想藉由投資韓國電池供應鏈，以獲得美國IRA法案補貼\n\n原文連結：\nhttps://...,中國擁有剛果的鈷礦和印度尼西亞的鎳礦等海外採礦業務,"{'情緒': '正向', '電動車品牌': '無'}"
5417,https://www.ptt.cc/bbs/car/M.1688023859.A.0D9....,[新聞]馬斯克一語成讖！美新創電動車申請破產,2023-06-29,原文連結：\nhttps://auto.ltn.com.tw/news/23176/\n特斯...,2020 年正式上市,"{'情緒': '中性', '電動車品牌': '無'}"
23954,https://www.ptt.cc/bbs/car/M.1701002913.A.7ED....,[心得]Mercedes-BenzW206C180一年駕駛心得,2023-11-26,（文長）\n從分享開箱心得到現在，陸續都有收到版友詢問使用上的心得，\n所以除了想分享一年多...,100%市區不大腳行駛下大約是10KM/L,"{'情緒': '中性', '電動車品牌': '無'}"
10850,https://www.ptt.cc/bbs/car/M.1691550798.A.A5D....,Re:[討論]高里程該選油車還是電車?,2023-08-09,我是油車直接換電車，因公司的車位是機械，所以當時選的是MODEL 3，開了八個月，\n\n簡...,有些人算到一年差幾萬元,"{'情緒': '中性', '電動車品牌': '無'}"


將LLM判斷的情緒挑出來

In [ ]:
sampled_5article_df['sentiment'] = sampled_5article_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)
sampled_df['sentiment'] = sampled_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)
sampled_df.head()

<ipython-input-15-ab83c6c2649f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_5article_df['sentiment'] = sampled_5article_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)


,artUrl,artTitle,artDate,artContent,sentence,LLM,sentiment
5339,https://www.ptt.cc/bbs/car/M.1687881934.A.112....,[問題]RX350&RX350h抉擇的兩難,2023-06-28,文章在01發文過，票數相當各有擁護者\n想再問問板上的各位前輩意見（鞠躬）\n\n上個月台股...,車室本來就很好,"{'情緒': '正向', '電動車品牌': '無'}",正向
3010,https://www.ptt.cc/bbs/car/M.1686196971.A.73C....,Re:[問題]Nx200350FSprotMODELY,2023-06-08,您列的這幾部車都是好車，但如果硬要比的話，以使用者過來人的經驗提供給您參考。\n\n以下是我...,遠端APP可控制車輛的的便利性,"{'情緒': '正向', '電動車品牌': '無'}",正向
15684,https://www.ptt.cc/bbs/car/M.1696528615.A.91E....,[新聞]幾乎一統美國充電江山！現代與KIA加入特,2023-10-06,原文連結：\nhttps://udn.com/news/amp/story/6811/748...,畢竟在美國特斯拉真的遠超越其他牌,"{'情緒': '正向', '電動車品牌': '特斯拉'}",正向
41309,https://www.ptt.cc/bbs/car/M.1710066962.A.A93....,[情報]趙小蘭妹妹生前駕特斯拉墜池塘狂求救,2024-03-10,好可怕怎開特斯拉會這個樣子\n一台兩百多萬到四百多萬的頂級豪華電車耶\n如果她開的是傳統汽油...,享年50歲,"{'情緒': '中性', '電動車品牌': '無'}",中性
15327,https://www.ptt.cc/bbs/car/M.1696396176.A.E42....,Re:[討論]補助60年換來一家代理中國汽車的公司,2023-10-04,:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n:\n政...,法國人可以買走,"{'情緒': '中性', '電動車品牌': '無'}",中性


隨機取50個句子的判斷結果

In [ ]:
sampled_df['sentiment'].value_counts()

sentiment
中性    26
正向    13
負向     9
Name: count, dtype: int64

隨機取5篇文章的判斷結果

In [ ]:
sampled_5article_df

,artUrl,artTitle,artDate,artContent,sentence,LLM,sentiment
5040,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,我朋友以前常常去水牛公園超級充電超級充電,"{'情緒': '中性', '電動車品牌': '無'}",中性
5041,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,大概750元,"{'情緒': '中性', '電動車品牌': '無'}",中性
5042,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,太貴,"{'情緒': '負向', '電動車品牌': '無'}",負向
5043,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,在嘉義火車站文化園區充電10小時400元還是太貴,"{'情緒': '負向', '電動車品牌': '無'}",負向
5044,https://www.ptt.cc/bbs/car/M.1687751424.A.66A....,Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,2023-06-26,我朋友以前常常去水牛公園超級充電\n超級充電。大概750元。太貴\n\n在嘉義火車站文化園區...,後來我朋友就發現文化創意產業園區有消費的話停車費半價,"{'情緒': '正向', '電動車品牌': '無'}",正向
...,...,...,...,...,...,...,...
43560,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,包含電動車組件跟相關系統,"{'情緒': '中性', '電動車品牌': '無'}",中性
43561,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,粉專有預先登記入場的連結,"{'情緒': '中性', '電動車品牌': '無'}",中性
43562,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,到時候可以掃QR code入場 馳諾瓦(飛宏子公司),"{'情緒': '中性', '電動車品牌': '飛宏子公司'}",中性
43563,https://www.ptt.cc/bbs/car/M.1710919026.A.D91....,[討論]4/17-4/20國際智慧移動展,2024-03-20,4月除了汽車零配件展，\n還有一個2035 E-Mobility Taiwan\n其實就電動...,OTA部分科絡達好似蠻強大,"{'情緒': '正向', '電動車品牌': '無'}",正向


5篇文章的每個句子的情緒分數占比

In [ ]:
# 計算每個 artTitle 中 sentiment 的數量
sentiment_counts = sampled_5article_df.groupby('artTitle')['sentiment'].value_counts().unstack(fill_value=0)

# 顯示結果
print(sentiment_counts)

# 如果要轉換為想要的格式，例如文章a 正向:5 負向:6 中性:10 無:0
formatted_result = sentiment_counts.reset_index().melt(id_vars='artTitle', value_vars=sentiment_counts.columns)
formatted_result.columns = ['artTitle', 'sentiment', 'count']
formatted_result = formatted_result.pivot(index='artTitle', columns='sentiment', values='count').fillna(0).astype(int)

# 顯示結果
formatted_result


sentiment                  不明  中性  正向  無  負向
artTitle                                    
Re:[新聞]台中充電樁將收費　電動車主鑽漏洞！私   1  39  14  0  30
Re:[討論]D卡熱門：試乘到被扣牌          0  15   4  1   5
[問題]賓士W206外匯車不香了嗎？          0   1   2  0   3
[市調]汽車市場研究活動車馬費800~6000元    0  34   4  0   0
[討論]4/17-4/20國際智慧移動展        0   7   1  0   0


sentiment,不明,中性,正向,無,負向
artTitle,,,,,
Re:[新聞]台中充電樁將收費 電動車主鑽漏洞！私,1,39,14,0,30
Re:[討論]D卡熱門：試乘到被扣牌,0,15,4,1,5
[問題]賓士W206外匯車不香了嗎？,0,1,2,0,3
[市調]汽車市場研究活動車馬費800~6000元,0,34,4,0,0
[討論]4/17-4/20國際智慧移動展,0,7,1,0,0


可以發現第一篇文章前面是在討論充電樁的方便性跟優惠，後面是在簡述新聞中個人的違規事由。所以正負向都有。



# 2. Prompt engineering
    1. One-shot
    2. Few-shot
    3. Chain of thought(CoT)
    4. Self-consistency

## 2-1. One-shot

In [ ]:
normal_template = """<start_of_turn>user
你是一位數學家，將會幫助回答數學問題。

問題：{question}<end_of_turn>
<start_of_turn>model
答案："""

normal_prompt = PromptTemplate.from_template(normal_template)

normal_chain = (
    {"question": RunnablePassthrough()}
    | normal_prompt
    | llm
)
normal_chain.invoke("一個長方形的周長是 24 厘米，長比寬多 2 厘米，求長和寬？")

'長 7 厘米，寬 5 厘米。\n\n解法：\n\n設長方形的長為 x 厘米，寬為 y 厘米。\n\n周長公式：周長 = 2（長 + 寬）\n\n根據問題，周長為 24 厘米，所以有：\n\n2（x + y）= 24\n\n解得：x + y = 12\n\n長比寬多 2 厘米，所以有：\n\nx = y + 2\n\n將 x 的式代入上一個公式中：\n\ny + 2 + y = 12\n\n解得：2y = 10\n\n解得：y = 5 厘米\n\n將 y 的值代入 x 的式中：\n\nx = 5 + 2 = 7 厘米\n\n因此，長方形的長為 7 厘米，寬為 5 厘米。'

One-shot結果正確。(可能是因為題目數字小，難度低，也可能純粹就是模型很厲害)

## 2-2. Few-shot

In [ ]:
fewshot_template = """<start_of_turn>user
你是一位數學家，將會幫助回答數學問題。

以下為一些例子：
```
問題：一個長方形的周長是 24 厘米，長比寬多 2 厘米，求長和寬？
答案：5跟7

問題：一個長方形的周長是 54 厘米，長比寬多 10 厘米，求長和寬？
答案：10跟7
```

現在請回答以下問題
問題：{question}<end_of_turn>
<start_of_turn>model
答案："""

fewshot_prompt = PromptTemplate.from_template(fewshot_template)

fewshot_chain = (
    {"question": RunnablePassthrough()}
    | fewshot_prompt
    | llm
)
fewshot_chain.invoke("一個長方形的周長是 82 厘米，長比寬多 12 厘米，求長和寬？")

'22跟10\n\n解答步驟：\n\n1. 設長和寬的長度為 x 和 y 厘米。\n2. 周長等式：2(x + y) = 82 厘米。\n3. 長比寬多等式：x - y = 12 厘米。\n4. 解解得：x = 12 + y。\n5. 將 x 的值代入周長等式中：2(12 + y + y) = 82 厘米。\n6. 解解得：2y + 24 = 41 厘米。\n7. 解解得：2y = 17 厘米。\n8. 解得：y = 8.5 厘米。\n9. 將 y 的值代入長比寬多等式中：x - 8.5 = 12 厘米。\n10. 解得：x = 20.5 厘米。\n\n因此，長和寬分別為 **22 厘米** 和 **10 厘米**。'

Few-shot回答錯誤。

## 2-3. CoT

In [ ]:
CoT_template = """<start_of_turn>user
你是一位數學家，將會幫助回答數學問題。

以下為一些例子：
```
例子1:
問題： 一個長方形的周長是 24 厘米，長比寬多 2 厘米，求長和寬。

Chain of Thought：

設長方形的寬為 x，則長為 x + 2。
長方形的周長公式為 2(長 + 寬)，即 2(x + (x + 2)) = 24。
展開並簡化，得到 2(2x + 2) = 24，進一步簡化得到 4x + 4 = 24。
將 4 移到等號的另一邊，得到 4x = 20。
將等式兩邊同時除以 4，得到 x = 5。
因此，寬為 5 厘米，長為 x + 2 = 7 厘米。
所以，長方形的長和寬分別是 7 厘米和 5 厘米。


例子2:
問題： 一個長方形的周長是 52 厘米，長比寬多 4 厘米，求長和寬。

Chain of Thought：

設長方形的寬為 x，則長為 x + 4。
長方形的周長公式為 2(長 + 寬)，即 2(x + (x + 4)) = 52。
展開並簡化，得到 2(2x + 4) = 52，進一步簡化得到 4x + 8 = 52。
將 4 移到等號的另一邊，得到 4x = 44。
將等式兩邊同時除以 4，得到 x = 11。
因此，寬為 11 厘米，長為 x + 4 = 15 厘米。
所以，長方形的長和寬分別是 11 厘米和 15 厘米。
```

現在請回答以下問題
問題：{question}<end_of_turn>
<start_of_turn>model
答案："""

CoT_prompt = PromptTemplate.from_template(CoT_template)

CoT_chain = (
    {"question": RunnablePassthrough()}
    | CoT_prompt
    | llm
)
CoT_chain.invoke("一個長方形的周長是 512 厘米，長比寬多 128 厘米，求長和寬？")

'長和寬分別是 **256 厘米** 和 **128 厘米**。\n\nChain of Thought：\n\n設長方形的寬為 x，則長為 x + 128。\n長方形的周長公式為 2(長 + 寬)，即 2(x + (x + 128)) = 512。\n展開並簡化，得到 2(2x + 128) = 512，進一步簡化得到 4x + 256 = 512。\n將 256 移到等號的另一邊，得到 4x = 256。\n將等式兩邊同時除以 4，得到 x = 64。\n因此，寬為 64 厘米，長為 x + 128 = 192 厘米。\n所以，長方形的長和寬分別是 **192 厘米** 和 **64 厘米**。'

CoT回答正確。

## 2-4. Self-consistency

In [ ]:
for i in range(1, 6):
  print(f"\n\n========== Run {i} ==========\n\n")
  print(CoT_chain.invoke("一個長方形的周長是 1024 厘米，長比寬多 256 厘米，求長和寬？"))



========== Run 1 ==========


長1024 厘米，寬768 厘米。

Chain of Thought：

設長方形的寬為 x，則長為 x + 256。
長方形的周長公式為 2(長 + 寬)，即 2(x + (x + 256)) = 1024。
展開並簡化，得到 2(2x + 256) = 1024，進一步簡化得到 4x + 512 = 1024。
將 512 移到等號的另一邊，得到 4x = 512。
將等式兩邊同時除以 4，得到 x = 128。
因此，寬為 128 厘米，長為 x + 256 = 384 厘米。
所以，長方形的長和寬分別是 384 厘米和 128 厘米。


========== Run 2 ==========


長 768 厘米，寬 512 厘米。

Chain of Thought：

設長方形的寬為 x，則長為 x + 256。
長方形的周長公式為 2(長 + 寬)，即 2(x + (x + 256)) = 1024。
展開並簡化，得到 2(2x + 256) = 1024，進一步簡化得到 4x + 512 = 1024。
將 512 移到等號的另一邊，得到 4x = 512。
將等式兩邊同時除以 4，得到 x = 128。
因此，寬為 128 厘米，長為 x + 256 = 384 厘米。
所以，長方形的長和寬分別是 384 厘米和 128 厘米。


========== Run 3 ==========


長 1024 厘米，寬 768 厘米。

Chain of Thought：

設長方形的寬為 x，則長為 x + 256。
長方形的周長公式為 2(長 + 寬)，即 2(x + (x + 256)) = 1024。
展開並簡化，得到 2(2x + 256) = 1024，進一步簡化得到 4x + 512 = 1024。
將 512 移到等號的另一邊，得到 4x = 512。
將等式兩邊同時除以 4，得到 x = 128。
因此，寬為 128 厘米，長為 x + 256 = 384 厘米。
所以，長方形的長和寬分別是 384 厘米和 128 厘米。


========== Run 4 ==========


長為 128 厘米，寬為 102 厘米。

Chain of Thought：



在run 1就回答正確了，後面也一樣的答案。

# 3. Retrieve QA (RAG)

這邊的資料是採用之前學校寄的「桃園國際機場股份有限公司113年大專院校暑期實習實施計畫」PDF


- 讀入 PDF 檔
- 切分檔案 chunk

In [ ]:
loader = PyPDFLoader("./data/113年大學院校暑期實習實施計畫.pdf")
pages = loader.load()
pages
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(pages)
texts[:10]

[Document(page_content='桃園國際機場股份有限公司 \n113年大專院校暑期實習實施計畫 \n  \n1 \n 一、 實施目的： \n為提升本國國際機場服務品質及提前培訓機場營運管理相關人員，以提升\n本國國際機場優勢競爭力，提供各大學院校學生學習，並實際瞭解國際機\n場運作情形，增進理論與實務之結合，特訂定本計畫。 \n二、 實施對象： \n以培養機場營運管理實務相關人才為目標，並依教育部統計處大專院校學科\n標準分類以下學類大三以上學生為本計畫申請對象。 \n實習類別 相關單位 科系學類要求 人數 \n（一）機場\n營運及管理\n實務 業務處 \n營運安全處 \n營運控制中心 \n航務處 \n物業開發處 \n 運輸管理相關科系尤佳 6 \n（二）機場\n工程及設施\n維護實務 工程處 \n專案工程處 \n維護處 \n資通處 \n職業安全衛生考核處 工程相關學系及資訊相關\n學系尤佳 6', metadata={'source': './data/113年大學院校暑期實習實施計畫.pdf', 'page': 0}),
 Document(page_content='桃園國際機場股份有限公司 \n113年大專院校暑期實習實施計畫 \n  \n2 \n 實習類別 相關單位 科系學類要求 人數 \n（三）企業\n運作實務 企業發展處 \n公共事務暨行銷處 \n財務處 \n會計處 \n人力資源處 \n政風處 \n總務處 \n法律事務處 \n總管理處 \n稽核室 不限科系 8 \n小計 20 \n(學生實習單位將由公司依業務需要及參考學生個人專業與意願指派) \n三、 辦理方式： \n(一) 由本公司公告本次實習實施計畫及提供實習名額，由各校推薦適合\n學生名單。 \n(二) 面試後就錄取名單通知各院校，並由各校轉知錄取學生依本公司所\n定時間至公司進行實習。 \n(三) 實習期間按月發給基本工資 2萬7,470元，並投保勞保及提撥勞工\n退休金。 \n(四) 本公司於實習期間不提供膳宿、交通工具及其他生活必需事項，均', metadata={'source': './data/113年大學院校暑期實習實施計畫.pdf', 'page': 1}),
 Document(page_content='桃園國際機場股份有限公司 \n113年大專院校暑期實習實

- 將文檔轉換為 embedding
- 存入 vectorDB

In [ ]:

embedding_function = SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding")

vectorstore = Chroma.from_documents(documents=texts, embedding=embedding_function)

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

測試找尋相關文檔

In [ ]:
# query it
query = "公司實習時間是哪時候？"
docs = vectorstore.similarity_search(query)

# print results
print(docs[0].page_content)

桃園國際機場股份有限公司 
113年大專院校暑期實習實施計畫 
  
4 
 七、 實習規劃： 
參與本公司及實習單位規劃之各項實作、訓練課程及活動。 
(一) 人力資源處：規劃共通性訓練課程，約1-2週。 
(二) 實習單位：依單位業務規劃專業課程或活動，約4-5週。 
(三) 期末專題發表：於實習最後一週舉行。 
八、 實習考核及評分標準：由實習單位主管負責評核實習成績，達70分以  
  上者得發給實習證明： 
(一) 實習期間表現（ 60％）依下列三項綜合評量 
1. 出勤狀況（ 20％）。 
2. 學習態度及敬業精神（20％）。 
3. 團隊精神及人際互動（20％）。 
(二) 實習書面報告（20％）。 
(三) 期末專題競賽（20％）。 
九、 其他事項： 
(一) 實習期間請事、病假依本公司請假規則、勞動基準法及性別工作平 
等法等相關法令規定辦理。 
(二) 差勤管理 
 不得無故缺席或遲到早退，並應遵守以下規定： 
1. 實習時間：依本公司實習單位上班時間為主。 
2. 請假規則：因故不克出席者，應先徵得實習單位同意，並填具請
假單。 
 病假：當日應向實習單位主管報備，並於3日內補辦請假手續。


上面沒有抓到正確的實習期間，但可以看到出來的這段裡面有包含多次實習時間、期間的字詞。

定義 RAG prompt 與 chain

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_prompt_templete = """<start_of_turn>user
"角色": "實習機器人",
"服務內容":
你是一個樂於幫助學生的機器人，關於實習計畫的問題你都可以依照以下文件中的規定來回答學生。\n
現在給予以下相關文件, 你將會基於以下 [文件][文件] 中的相關文件與 [回答規則][回答規則] 中的回答規則來回答學生的問題。\n
\n
[文件]
{context}
[文件]
\n
[回答規則]
只依據 [文件][文件] 中提供的資訊來回答，並且遵照下列的幾點規則：
- 不撒謊或幻想 [文件][文件] 中未明確提供的答案
- 如果不確定答案或答案未明確包含在 [文件][文件] 中，請回答：“我很抱歉，我不知道如何提供幫助。”
- 回答簡短、相關且簡潔的答案
- 回答字數請少於 150 個字
- 請在 [結果]: 之後回答你的答案
[回答規則]

<</SYS>>
現在，基於上述的文件與規則，回答以下 () 中的問題\n[學生]:({question})\n<end_of_turn>
<start_of_turn>model[結果]:"""

rag_prompt = PromptTemplate.from_template(rag_prompt_templete)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)
rag_chain.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                    ****                ****                
                 ***                        ***             
               **                              ***          
+----------------------+                          **        
| VectorStoreRetriever |                           *        
+----------------------+                           *        
            *                                      *        
            *                                      *        
            *                                      *        
+---------------------+                     +-------------+ 
| Lambda(format_docs) |                     | Passthrough | 
+---------------------+                     +-------------+ 
                    ****                ****                
                        

In [ ]:
rag_chain.invoke("實習每個月可以領多少錢？")


' 實習期間按月發給基本工資 2 萬 7,470 元。'

In [ ]:
rag_chain.invoke("面談時間是哪時候？")

' 面試時間是 113年06月7日前。'

In [ ]:
rag_chain.invoke("實習期間落在幾月幾號到幾月幾號？")

' 文件並未包含實習期間的具體日期，無法回答。'

實際對照PDF檢查後，除了實習期間外，其他回答正確。